In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn import neighbors
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
Y = car_evaluation.data.targets 


rows_to_drop = X.isna().any(axis=1)
Y = Y[~rows_to_drop]
X = X.dropna()
class_mapping = {"unacc": -1, "acc": -1, "good": 1, "vgood": 1}
Y['class'] = Y['class'].map(class_mapping)
buying_mapping = {'low': 1, "med": 2, "high": 3, "vhigh": 4}
X['buying'] = X['buying'].map(buying_mapping)
X['maint'] = X['maint'].map(buying_mapping)
door_mapping = {'1': 1, "2": 2, "3": 3, "4": 4, "5more":5}
X['doors'] = X['doors'].map(door_mapping)
person_mapping = {"2": 2, "4": 4, "more":5}
X['persons'] = X['persons'].map(person_mapping)
lug_mapping = {'small': 1, "med": 2, "big": 3}
X['lug_boot'] = X['lug_boot'].map(lug_mapping)
safety_mapping = {'low': 1, "med": 2, "high": 3}
X['safety'] = X['safety'].map(safety_mapping)
X

,buying,maint,doors,persons,lug_boot,safety
0,4,4,2,2,1,1
1,4,4,2,2,1,2
2,4,4,2,2,1,3
3,4,4,2,2,2,1
4,4,4,2,2,2,2
...,...,...,...,...,...,...
1723,1,1,5,5,2,2
1724,1,1,5,5,2,3
1725,1,1,5,5,3,1
1726,1,1,5,5,3,2


In [3]:
def calc_error(X, Y, classifier):
    
    # Hint: Use classifier.predict()
    Y_pred = classifier.predict(X)

    # Hint: Use accuracy_score().
    precision = accuracy_score(Y, Y_pred)
    recall = recall_score(Y, Y_pred)
    f_value = (2 * precision * recall) / (precision + recall)
    return precision, recall, f_value

In [4]:
def SVM(X_train, Y_train):
    C_list = [0.0001, 0.001, 0.1, 1, 10, 100, 1000]
    opt_validation = 0.0
    opt_C          = None  # Optimal C.

    for C in C_list:
        classifier = svm.LinearSVC(penalty = 'l2', loss = 'hinge', max_iter = 100000, C = C)
    
        cv_results = cross_validate(classifier, X_train, Y_train)
        validation_accuracy = np.mean(cv_results['test_score'])
    
        # Judge if it is the optimal one.
        if validation_accuracy > opt_validation:
            opt_validation = validation_accuracy
            opt_classifier = classifier
            opt_C = C
            
    
    opt_classifier = svm.LinearSVC(penalty = 'l2', loss = 'hinge', max_iter = 100000, C = opt_C)
    opt_classifier.fit(X_train, Y_train)
    opt_error = calc_error(X_train, Y_train, opt_classifier)
    
    test_error = calc_error(X_test, Y_test, opt_classifier)

    print('Best parameter C* = {}'.format(opt_C))
    print('Training accuracy: {}'.format(opt_error[0]))
    print('Training recall: {}'.format(opt_error[1]))
    print('Training F-value: {}'.format(opt_error[2]))
    
    print('Validation accuracy: {}'.format(opt_validation))

    print('Testing accuracy: {}'.format(test_error[0]))
    print('Testing recall: {}'.format(test_error[1]))
    print('Testing F-value: {}'.format(test_error[2]))

In [5]:
def KNN(X_train, Y_train):
    opt_validation = 0.0
    opt_n          = None  # Optimal C.
    n_list = [3,4,5,6,7,8]

    for n in n_list:
        classifier = neighbors.KNeighborsClassifier(n_neighbors = n)
    
        cv_results = cross_validate(classifier, X_train, Y_train)
        validation_accuracy = np.mean(cv_results['test_score'])
    
    
        # Judge if it is the optimal one.
        if validation_accuracy > opt_validation:
            opt_validation = validation_accuracy
            opt_classifier = classifier
            opt_n = n
            
    
    opt_classifier = neighbors.KNeighborsClassifier(n_neighbors = opt_n)
    opt_classifier.fit(X_train, Y_train)
    opt_error = calc_error(X_train, Y_train, opt_classifier)
    
    test_error = calc_error(X_test, Y_test, opt_classifier)

    print('Best parameter C* = {}'.format(opt_n))
    print('Training accuracy: {}'.format(opt_error[0]))
    print('Training recall: {}'.format(opt_error[1]))
    print('Training F-value: {}'.format(opt_error[2]))
    
    print('Validation accuracy: {}'.format(opt_validation))

    print('Testing accuracy: {}'.format(test_error[0]))
    print('Testing recall: {}'.format(test_error[1]))
    print('Testing F-value: {}'.format(test_error[2]))

In [6]:
def LogisticRegression(X_train, Y_train):
    C_list = [0.0001, 0.001, 0.1, 1, 10, 100, 1000]
    opt_validation = 0.0
    opt_C          = None  # Optimal C.

    for C in C_list:
        classifier = linear_model.LogisticRegression(penalty = 'l2', C = C)
    
        cv_results = cross_validate(classifier, X_train, Y_train)
        validation_accuracy = np.mean(cv_results['test_score'])
    
        # Judge if it is the optimal one.
        if validation_accuracy > opt_validation:
            opt_validation = validation_accuracy
            opt_classifier = classifier
            opt_C = C
            
    
    opt_classifier = linear_model.LogisticRegression(penalty = 'l2', C = C)
    opt_classifier.fit(X_train, Y_train)
    opt_error = calc_error(X_train, Y_train, opt_classifier)
    
    test_error = calc_error(X_test, Y_test, opt_classifier)

    print('Best parameter C* = {}'.format(opt_C))
    print('Training accuracy: {}'.format(opt_error[0]))
    print('Training recall: {}'.format(opt_error[1]))
    print('Training F-value: {}'.format(opt_error[2]))
    
    print('Validation accuracy: {}'.format(opt_validation))

    print('Testing accuracy: {}'.format(test_error[0]))
    print('Testing recall: {}'.format(test_error[1]))
    print('Testing F-value: {}'.format(test_error[2]))

# 20/80 Partition
# Seed 1

In [7]:
# Seed 1, 20/80 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2, random_state=1)

In [8]:
# car_eval svm
# Partition: [20, 80]
# Seed 1
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

Best parameter C* = 100
Training accuracy: 0.9768115942028985
Training recall: 0.8181818181818182
Training F-value: 0.8904873752201996
Validation accuracy: 0.9768115942028986
Testing accuracy: 0.9522776572668112
Testing recall: 0.7227722772277227
Testing F-value: 0.8218022361267823


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

In [9]:
# car_eval KNN
# Partition: [20, 80]
# Seed 1
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9768115942028985
Training recall: 0.8181818181818182
Training F-value: 0.8904873752201996
Validation accuracy: 0.9623188405797102
Testing accuracy: 0.9689081706435285
Testing recall: 0.7623762376237624
Testing F-value: 0.8533231885076769


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

In [10]:
# car_eval LR
# Partition: [20, 80]
# Seed 1
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best parameter C* = 1
Training accuracy: 0.9710144927536232
Training recall: 0.8181818181818182
Training F-value: 0.8880706921944036
Validation accuracy: 0.9594202898550724
Testing accuracy: 0.9573391178597253
Testing recall: 0.7821782178217822
Testing F-value: 0.8609397442598392


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 20/80 Partition
# Seed 2

In [11]:
# Seed 2, 20/80 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2, random_state=2)

In [12]:
# car_eval svm
# Partition: [20, 80]
# Seed 2
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = c

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

Best parameter C* = 1000
Training accuracy: 0.9565217391304348
Training recall: 0.7419354838709677
Training F-value: 0.8356729975227085
Validation accuracy: 0.9478260869565217
Testing accuracy: 0.9674620390455532
Testing recall: 0.7475728155339806
Testing F-value: 0.8434211334192915


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [13]:
# car_eval KNN
# Partition: [20, 80]
# Seed 2
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9768115942028985
Training recall: 0.8387096774193549
Training F-value: 0.9025081114487304
Validation accuracy: 0.9652173913043478
Testing accuracy: 0.9783080260303688
Testing recall: 0.8058252427184466
Testing F-value: 0.8837291656829879


In [14]:
# car_eval LR
# Partition: [20, 80]
# Seed 2
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best parameter C* = 100
Training accuracy: 0.9565217391304348
Training recall: 0.7096774193548387
Training F-value: 0.8148148148148149
Validation accuracy: 0.9536231884057971
Testing accuracy: 0.9689081706435285
Testing recall: 0.7184466019417476
Testing F-value: 0.8250888243565954


# 20/80 Partition
# Seed 3

In [15]:
# Seed 1, 20/80 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2, random_state=3)

In [16]:
# car_eval svm
# Partition: [20, 80]
# Seed 3
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = c

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

Best parameter C* = 1000
Training accuracy: 0.9594202898550724
Training recall: 0.78125
Training F-value: 0.8612166311078733
Validation accuracy: 0.9391304347826088
Testing accuracy: 0.9631236442516269
Testing recall: 0.7745098039215687
Testing F-value: 0.8585800482210826


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [17]:
# car_eval KNN
# Partition: [20, 80]
# Seed 3
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 5
Training accuracy: 0.991304347826087
Training recall: 0.9375
Training F-value: 0.963651732882502
Validation accuracy: 0.9623188405797102
Testing accuracy: 0.9674620390455532
Testing recall: 0.7647058823529411
Testing F-value: 0.8542173112338857


In [18]:
# car_eval LR
# Partition: [20, 80]
# Seed 3
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best parameter C* = 1
Training accuracy: 0.9623188405797102
Training recall: 0.6875
Training F-value: 0.802020423849786
Validation accuracy: 0.9507246376811596
Testing accuracy: 0.9681851048445409
Testing recall: 0.7254901960784313
Testing F-value: 0.829449188012724


# 50/50 Partition
# Seed 1

In [19]:
# Seed 1, 50/50 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.5, random_state=1)

In [20]:
# car_eval svm
# Partition: [50, 50]
# Seed 1
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

Best parameter C* = 10
Training accuracy: 0.9664351851851852
Training recall: 0.7391304347826086
Training F-value: 0.8376360900480925
Validation accuracy: 0.9652708697405565
Testing accuracy: 0.9675925925925926
Testing recall: 0.676923076923077
Testing F-value: 0.796569795140543


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [21]:
# car_eval KNN
# Partition: [50, 50]
# Seed 1
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9780092592592593
Training recall: 0.8405797101449275
Training F-value: 0.904101755259784
Validation accuracy: 0.9699018685307165
Testing accuracy: 0.9791666666666666
Testing recall: 0.7692307692307693
Testing F-value: 0.8615948670944088


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [22]:
# car_eval LR
# Partition: [50, 50]
# Seed 1
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Best parameter C* = 100
Training accuracy: 0.9675925925925926
Training recall: 0.7536231884057971
Training F-value: 0.8473083070206214
Validation accuracy: 0.9698951471972039
Testing accuracy: 0.9664351851851852
Testing recall: 0.6923076923076923
Testing F-value: 0.8067199828243251


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 50/50 Partition
# Seed 2

In [23]:
# Seed 2, 50/50 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.5, random_state=2)

In [24]:
# car_eval svm
# Partition: [50, 50]
# Seed 2
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

Best parameter C* = 100
Training accuracy: 0.9675925925925926
Training recall: 0.7352941176470589
Training F-value: 0.8355989125219894
Validation accuracy: 0.9687457991665547
Testing accuracy: 0.9652777777777778
Testing recall: 0.6818181818181818
Testing F-value: 0.7991567650440782


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [25]:
# car_eval KNN
# Partition: [50, 50]
# Seed 2
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 5
Training accuracy: 0.9895833333333334
Training recall: 0.8823529411764706
Training F-value: 0.9328968903436988
Validation accuracy: 0.9710512165613657
Testing accuracy: 0.9756944444444444
Testing recall: 0.7272727272727273
Testing F-value: 0.8333642261353105


In [26]:
# car_eval LR
# Partition: [50, 50]
# Seed 2
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Best parameter C* = 10
Training accuracy: 0.9699074074074074
Training recall: 0.7794117647058824
Training F-value: 0.864287382268234
Validation accuracy: 0.9687390778330421
Testing accuracy: 0.9675925925925926
Testing recall: 0.6666666666666666
Testing F-value: 0.7894239848914071


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 50/50 Partition
# Seed 3

In [27]:
# Seed 3, 50/50 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.5, random_state=3)

In [28]:
# car_eval svm
# Partition: [50, 50]
# Seed 3
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/si

Best parameter C* = 10
Training accuracy: 0.9641203703703703
Training recall: 0.7164179104477612
Training F-value: 0.8220141237420721
Validation accuracy: 0.9629520096787203
Testing accuracy: 0.96875
Testing recall: 0.7611940298507462
Testing F-value: 0.8525208951199784


In [29]:
# car_eval KNN
# Partition: [50, 50]
# Seed 3
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9837962962962963
Training recall: 0.8656716417910447
Training F-value: 0.9209616857521811
Validation accuracy: 0.9745395886543889
Testing accuracy: 0.9745370370370371
Testing recall: 0.7611940298507462
Testing F-value: 0.854754274567567


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

In [30]:
# car_eval LR
# Partition: [50, 50]
# Seed 3
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Best parameter C* = 10
Training accuracy: 0.9664351851851852
Training recall: 0.7611940298507462
Training F-value: 0.8516233538981491
Validation accuracy: 0.9641013577093694
Testing accuracy: 0.9710648148148148
Testing recall: 0.7611940298507462
Testing F-value: 0.8534160375759148


# 80/20 Partition
# Seed 1

In [31]:
# Seed 1, 80/20 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=1)

In [32]:
# car_eval svm
# Partition: [80, 20]
# Seed 1
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/si

Best parameter C* = 100
Training accuracy: 0.9681620839363242
Training recall: 0.7570093457943925
Training F-value: 0.8496636718566254
Validation accuracy: 0.9667085229948202
Testing accuracy: 0.9595375722543352
Testing recall: 0.7037037037037037
Testing F-value: 0.8119449092547304


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [33]:
# car_eval KNN
# Partition: [80, 20]
# Seed 1
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 5
Training accuracy: 0.9862518089725036
Training recall: 0.8691588785046729
Training F-value: 0.9240105406262415
Validation accuracy: 0.9768377544079945
Testing accuracy: 0.9797687861271677
Testing recall: 0.7777777777777778
Testing F-value: 0.8671660880687009


In [34]:
# car_eval LR
# Partition: [80, 20]
# Seed 1
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best parameter C* = 100
Training accuracy: 0.9696092619392185
Training recall: 0.7663551401869159
Training F-value: 0.856083271004737
Validation accuracy: 0.9674331606759796
Testing accuracy: 0.9624277456647399
Testing recall: 0.7037037037037037
Testing F-value: 0.8129778348859622


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 80/20 Partition
# Seed 2

In [35]:
# Seed 2, 80/20 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=2)

In [36]:
# car_eval svm
# Partition: [80, 20]
# Seed 2
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was pas

Best parameter C* = 1000
Training accuracy: 0.967438494934877
Training recall: 0.7129629629629629
Training F-value: 0.82093217973765
Validation accuracy: 0.9674488567990374
Testing accuracy: 0.9653179190751445
Testing recall: 0.6923076923076923
Testing F-value: 0.8063304721030043


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [37]:
# car_eval KNN
# Partition: [80, 20]
# Seed 2
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9942112879884226
Training recall: 0.9444444444444444
Training F-value: 0.9686890888732219
Validation accuracy: 0.9783001098728613
Testing accuracy: 0.9855491329479769
Testing recall: 0.8076923076923077
Testing F-value: 0.8878006446813785


In [38]:
# car_eval LR
# Partition: [80, 20]
# Seed 2
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Best parameter C* = 100
Training accuracy: 0.9681620839363242
Training recall: 0.7592592592592593
Training F-value: 0.8510790139162542
Validation accuracy: 0.9688981321613561
Testing accuracy: 0.9682080924855492
Testing recall: 0.7307692307692307
Testing F-value: 0.8328971473436273


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 80/20 Partition
# Seed 3

In [39]:
# Seed 3, 80/20 partition

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=3)

In [40]:
# car_eval svm
# Partition: [80, 20]
# Seed 3
np.random.seed(1)
SVM(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the 

Best parameter C* = 100
Training accuracy: 0.9659913169319826
Training recall: 0.7410714285714286
Training F-value: 0.8387138283136406
Validation accuracy: 0.9623737770104117
Testing accuracy: 0.9710982658959537
Testing recall: 0.7727272727272727
Testing F-value: 0.8606298026216663


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [41]:
# car_eval KNN
# Partition: [80, 20]
# Seed 3
KNN(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

Best parameter C* = 7
Training accuracy: 0.9927641099855282
Training recall: 0.9285714285714286
Training F-value: 0.9595954161510734
Validation accuracy: 0.9840867472400984
Testing accuracy: 0.9826589595375722
Testing recall: 0.7727272727272727
Testing F-value: 0.8651399491094147


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [42]:
# car_eval LR
# Partition: [80, 20]
# Seed 3
LogisticRegression(X_train, Y_train)

/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Best parameter C* = 1
Training accuracy: 0.9638205499276411
Training recall: 0.7232142857142857
Training F-value: 0.8263596884262769
Validation accuracy: 0.9638204363522209
Testing accuracy: 0.9710982658959537
Testing recall: 0.7727272727272727
Testing F-value: 0.8606298026216663


/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sblevin/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s